## Import Library

In [1]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 97.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [2]:
!pip install requests

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import pickle
import math
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Preprocessing

### Load Data

In [4]:
train = pd.read_csv('https://raw.githubusercontent.com/jryand25/datathon-2023/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/jryand25/datathon-2023/main/test.csv')

### Check Null

In [5]:
print(train.isnull().sum())

waktu_setempat      0
id_jalan            0
id_titik_mulai      0
id_titik_akhir      0
rerata_kecepatan    0
dtype: int64


### Check Duplicate

In [6]:
duplicates = train.duplicated()

print(len(train[duplicates]))

0


### Retrieve Data Jalan

In [7]:
def get_osm_way(way_id):
    # Overpass API endpoint URL
    overpass_url = "https://overpass-api.de/api/interpreter"

    # Overpass query to get way information
    way_query = f"""
    [out:json];
    way({way_id});
    out;
    """

    try:
        # Send POST requests to Overpass API for way and node data
        response_way = requests.post(overpass_url, data=way_query)

        # Parse the JSON responses
        data_way = response_way.json()
        data_way = data_way['elements'][0]['tags']
        highway = data_way['highway']
        lit = data_way['lit']
        max_speed = data_way['maxspeed']

        return highway, lit, max_speed

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

In [8]:
# way_data
way_id_distinct = sorted(train['id_jalan'].unique())
WAY_DISTINCT_DATA = {}

for way_id in way_id_distinct:
    data = get_osm_way(way_id)
    WAY_DISTINCT_DATA[way_id] = [data[0], data[1], data[2]]

In [9]:
# Define a function to look up values in WAY_DISTINCT_DATA
def get_highway(row):
    return WAY_DISTINCT_DATA[row['id_jalan']][0]
def get_lit(row):
    return WAY_DISTINCT_DATA[row['id_jalan']][1]
def get_maxspeed(row):
    return WAY_DISTINCT_DATA[row['id_jalan']][2]

### Retrieve Data Titik

In [10]:
def get_osm_node(node_id):
    # Overpass API endpoint URL
    overpass_url = "https://overpass-api.de/api/interpreter"

    # Overpass query to get node information
    node_query = f"""
    [out:json];
    node({node_id});
    out;
    """

    try:
        # Send POST requests to Overpass API for way and node data
        response_node = requests.post(overpass_url, data=node_query)

        # Parse the JSON responses
        data_node = response_node.json()
        if len(data_node['elements']) == 0:
            print("Elements is empty")
            return None, None
        lat = data_node['elements'][0]['lat']
        lon = data_node['elements'][0]['lon']
        return lat,lon

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None, None

Titik Awal

In [11]:
# way_data
awal_id_distinct = sorted(train['id_titik_mulai'].unique())
AWAL_ID_DISTINCT_DATA = {}

counter = 1
for awal_id in awal_id_distinct:
    data = get_osm_node(awal_id)
    AWAL_ID_DISTINCT_DATA[awal_id] = [data[0], data[1]] or [None, None]
    counter+=1

Elements is empty
Elements is empty
Elements is empty
Elements is empty
Elements is empty
Elements is empty
Elements is empty


In [12]:
# Define a function to look up values in WAY_DISTINCT_DATA
def get_lat_a(row):
    return AWAL_ID_DISTINCT_DATA[row['id_titik_mulai']][0]
def get_lon_a(row):
    return AWAL_ID_DISTINCT_DATA[row['id_titik_mulai']][1]

Titik Akhir

In [13]:
# way_data
akhir_id_distinct = sorted(train['id_titik_akhir'].unique())
AKHIR_ID_DISTINCT_DATA = {}

counter = 1
for akhir_id in akhir_id_distinct:
    data = get_osm_node(akhir_id)
    AKHIR_ID_DISTINCT_DATA[akhir_id] = [data[0], data[1]] or [None, None]
    counter+=1

Elements is empty
Elements is empty
Elements is empty
Elements is empty
Elements is empty
Elements is empty
Elements is empty


In [14]:
# Define a function to look up values in WAY_DISTINCT_DATA
def get_lat_b(row):
    return AKHIR_ID_DISTINCT_DATA[row['id_titik_akhir']][0]
def get_lon_b(row):
    return AKHIR_ID_DISTINCT_DATA[row['id_titik_akhir']][1]

### Start Pipeline

Load Data

In [15]:
train = pd.read_csv('https://raw.githubusercontent.com/jryand25/datathon-2023/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/jryand25/datathon-2023/main/test.csv')

In [16]:
train['highway'] = train.apply(get_highway, axis=1)
train['lit'] = train.apply(get_lit, axis=1)
train['max_speed'] = train.apply(get_maxspeed, axis=1)
train['mulai_lat'] = train.apply(get_lat_a, axis=1)
train['mulai_lon'] = train.apply(get_lon_a, axis=1)
train['akhir_lat'] = train.apply(get_lat_b, axis=1)
train['akhir_lon'] = train.apply(get_lon_b, axis=1)

In [17]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,lit,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126,trunk,yes,30 mph,51.434928,-0.161176,51.434975,-0.161090
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576,primary,yes,30 mph,51.623990,-0.176398,51.623811,-0.176424
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587,secondary,yes,20 mph,51.452530,-0.152437,51.452749,-0.151896
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063,primary,yes,20 mph,51.465851,-0.154188,51.465654,-0.155334
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336,primary,yes,30 mph,51.530289,-0.228343,51.530197,-0.227949


In [18]:
test['highway'] = test.apply(get_highway, axis=1)
test['lit'] = test.apply(get_lit, axis=1)
test['max_speed'] = test.apply(get_maxspeed, axis=1)
test['mulai_lat'] = test.apply(get_lat_a, axis=1)
test['mulai_lon'] = test.apply(get_lon_a, axis=1)
test['akhir_lat'] = test.apply(get_lat_b, axis=1)
test['akhir_lon'] = test.apply(get_lon_b, axis=1)

In [19]:
test.head()

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,highway,lit,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544,trunk,yes,30 mph,51.465182,-0.255240,51.465204,-0.254595
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496,trunk,yes,30 mph,51.458973,-0.141882,51.459254,-0.141646
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020,secondary,yes,20 mph,51.449437,-0.160688,51.449295,-0.161060
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489,trunk,yes,30 mph,51.457325,-0.143228,51.457466,-0.143110
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110,trunk,yes,30 mph,51.467314,-0.112842,51.466309,-0.113371


Drop Lit karena semua bernilai yes

In [20]:
train['lit'].value_counts()

yes    398648
Name: lit, dtype: int64

In [21]:
def drop_lit(data):
    data.drop(columns=['lit'], inplace=True)
    return data

In [22]:
train = drop_lit(train)
test = drop_lit(test)

In [23]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126,trunk,30 mph,51.434928,-0.161176,51.434975,-0.161090
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576,primary,30 mph,51.623990,-0.176398,51.623811,-0.176424
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587,secondary,20 mph,51.452530,-0.152437,51.452749,-0.151896
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063,primary,20 mph,51.465851,-0.154188,51.465654,-0.155334
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336,primary,30 mph,51.530289,-0.228343,51.530197,-0.227949


In [24]:
test.head()

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544,trunk,30 mph,51.465182,-0.255240,51.465204,-0.254595
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496,trunk,30 mph,51.458973,-0.141882,51.459254,-0.141646
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020,secondary,20 mph,51.449437,-0.160688,51.449295,-0.161060
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489,trunk,30 mph,51.457325,-0.143228,51.457466,-0.143110
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110,trunk,30 mph,51.467314,-0.112842,51.466309,-0.113371


Remove mph dari max_speed

In [25]:
def remove_mph(data):
    data['max_speed'] = data['max_speed'].str.replace(' mph', '')
    data['max_speed'] = np.array(data['max_speed']).astype(np.float64)
    return data

In [26]:
train = remove_mph(train)
test = remove_mph(test)

In [27]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126,trunk,30.0,51.434928,-0.161176,51.434975,-0.161090
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576,primary,30.0,51.623990,-0.176398,51.623811,-0.176424
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587,secondary,20.0,51.452530,-0.152437,51.452749,-0.151896
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063,primary,20.0,51.465851,-0.154188,51.465654,-0.155334
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336,primary,30.0,51.530289,-0.228343,51.530197,-0.227949


In [28]:
test.head()

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544,trunk,30.0,51.465182,-0.255240,51.465204,-0.254595
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496,trunk,30.0,51.458973,-0.141882,51.459254,-0.141646
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020,secondary,20.0,51.449437,-0.160688,51.449295,-0.161060
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489,trunk,30.0,51.457325,-0.143228,51.457466,-0.143110
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110,trunk,30.0,51.467314,-0.112842,51.466309,-0.113371


Labeling ID jalan

In [29]:
label_encoder = LabelEncoder()
train['id_jalan'] = label_encoder.fit_transform(train['id_jalan'])
test['id_jalan'] = label_encoder.transform(test['id_jalan'])

In [30]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,2020-02-01 01:00:00+00:00,19,21390008,1425033102,29.126,trunk,30.0,51.434928,-0.161176,51.434975,-0.161090
1,2020-02-01 01:00:00+00:00,12,1677092762,579493410,46.576,primary,30.0,51.623990,-0.176398,51.623811,-0.176424
2,2020-02-01 01:00:00+00:00,8,26486694,1930267566,36.587,secondary,20.0,51.452530,-0.152437,51.452749,-0.151896
3,2020-02-01 01:00:00+00:00,16,1111592522,3775231113,34.063,primary,20.0,51.465851,-0.154188,51.465654,-0.155334
4,2020-02-01 01:00:00+00:00,7,5940503398,5940503394,38.336,primary,30.0,51.530289,-0.228343,51.530197,-0.227949


In [31]:
test.head()

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,2020-02-23 00:00:00+00:00,3,32046542,6454026544,trunk,30.0,51.465182,-0.255240,51.465204,-0.254595
1,1,2020-02-23 00:00:00+00:00,18,1314925464,1314925496,trunk,30.0,51.458973,-0.141882,51.459254,-0.141646
2,2,2020-02-23 00:00:00+00:00,8,1482086782,26481020,secondary,20.0,51.449437,-0.160688,51.449295,-0.161060
3,3,2020-02-23 00:00:00+00:00,18,3892883,267337489,trunk,30.0,51.457325,-0.143228,51.457466,-0.143110
4,4,2020-02-23 00:00:00+00:00,14,266041030,2592978110,trunk,30.0,51.467314,-0.112842,51.466309,-0.113371


Labeling Highway

In [32]:
highway_mapping = {'secondary': 0, 'primary': 1, 'trunk': 2}
train['highway'] = train['highway'].map(highway_mapping)
test['highway'] = test['highway'].map(highway_mapping)

In [33]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,2020-02-01 01:00:00+00:00,19,21390008,1425033102,29.126,2,30.0,51.434928,-0.161176,51.434975,-0.161090
1,2020-02-01 01:00:00+00:00,12,1677092762,579493410,46.576,1,30.0,51.623990,-0.176398,51.623811,-0.176424
2,2020-02-01 01:00:00+00:00,8,26486694,1930267566,36.587,0,20.0,51.452530,-0.152437,51.452749,-0.151896
3,2020-02-01 01:00:00+00:00,16,1111592522,3775231113,34.063,1,20.0,51.465851,-0.154188,51.465654,-0.155334
4,2020-02-01 01:00:00+00:00,7,5940503398,5940503394,38.336,1,30.0,51.530289,-0.228343,51.530197,-0.227949


In [34]:
test.head()

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,2020-02-23 00:00:00+00:00,3,32046542,6454026544,2,30.0,51.465182,-0.255240,51.465204,-0.254595
1,1,2020-02-23 00:00:00+00:00,18,1314925464,1314925496,2,30.0,51.458973,-0.141882,51.459254,-0.141646
2,2,2020-02-23 00:00:00+00:00,8,1482086782,26481020,0,20.0,51.449437,-0.160688,51.449295,-0.161060
3,3,2020-02-23 00:00:00+00:00,18,3892883,267337489,2,30.0,51.457325,-0.143228,51.457466,-0.143110
4,4,2020-02-23 00:00:00+00:00,14,266041030,2592978110,2,30.0,51.467314,-0.112842,51.466309,-0.113371


Labeling ID Titik Mulai

In [35]:
label_encoder = LabelEncoder()
train['id_titik_mulai'] = label_encoder.fit_transform(train['id_titik_mulai'])
test['id_titik_mulai'] = label_encoder.transform(test['id_titik_mulai'])

Labeling ID Titik Akhir

In [36]:
label_encoder = LabelEncoder()
train['id_titik_akhir'] = label_encoder.fit_transform(train['id_titik_akhir'])
test['id_titik_akhir'] = label_encoder.transform(test['id_titik_akhir'])

In [37]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,2020-02-01 01:00:00+00:00,19,102,345,29.126,2,30.0,51.434928,-0.161176,51.434975,-0.161090
1,2020-02-01 01:00:00+00:00,12,378,279,46.576,1,30.0,51.623990,-0.176398,51.623811,-0.176424
2,2020-02-01 01:00:00+00:00,8,133,390,36.587,0,20.0,51.452530,-0.152437,51.452749,-0.151896
3,2020-02-01 01:00:00+00:00,16,329,423,34.063,1,20.0,51.465851,-0.154188,51.465654,-0.155334
4,2020-02-01 01:00:00+00:00,7,467,466,38.336,1,30.0,51.530289,-0.228343,51.530197,-0.227949


In [38]:
test.head()

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,2020-02-23 00:00:00+00:00,3,153,487,2,30.0,51.465182,-0.255240,51.465204,-0.254595
1,1,2020-02-23 00:00:00+00:00,18,342,343,2,30.0,51.458973,-0.141882,51.459254,-0.141646
2,2,2020-02-23 00:00:00+00:00,8,351,131,0,20.0,51.449437,-0.160688,51.449295,-0.161060
3,3,2020-02-23 00:00:00+00:00,18,68,243,2,30.0,51.457325,-0.143228,51.457466,-0.143110
4,4,2020-02-23 00:00:00+00:00,14,241,408,2,30.0,51.467314,-0.112842,51.466309,-0.113371


## Feature Engineering

Memisahkan Date Time

In [39]:
def create_date_time(data):
    time = None
    date = None
    data.insert(loc=data.columns.get_loc('waktu_setempat') + 1, column='date', value=date)
    data.insert(loc=data.columns.get_loc('waktu_setempat') + 2, column='time', value=time)
    return data

In [40]:
def split_date_time(data):
    data['waktu_setempat'] = pd.to_datetime(data['waktu_setempat'])

    data['date'] = data['waktu_setempat'].dt.date
    data['time'] = data['waktu_setempat'].dt.time

    data.drop(columns=['waktu_setempat'], inplace=True)
    return data

In [41]:
train = create_date_time(train)
train = split_date_time(train)

test = create_date_time(test)
test = split_date_time(test)

Mengubah Date menjadi Day

In [42]:
def create_day(data):
    day = None
    data.insert(loc=data.columns.get_loc('date') + 1, column='day', value=day)
    return data

In [43]:
def convert_date_to_day(data):
    # Convert the 'date' column to pandas datetime objects
    data['date'] = pd.to_datetime(data['date'])

    # Use the 'dt.weekday' property to get the day of the week as an integer
    data['day'] = data['date'].dt.weekday

    data.drop(columns=['date'], inplace=True)
    return data

In [44]:
train = create_day(train)
train = convert_date_to_day(train)

test = create_day(test)
test = convert_date_to_day(test)

In [45]:
train.head()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,5,01:00:00,19,102,345,29.126,2,30.0,51.434928,-0.161176,51.434975,-0.161090
1,5,01:00:00,12,378,279,46.576,1,30.0,51.623990,-0.176398,51.623811,-0.176424
2,5,01:00:00,8,133,390,36.587,0,20.0,51.452530,-0.152437,51.452749,-0.151896
3,5,01:00:00,16,329,423,34.063,1,20.0,51.465851,-0.154188,51.465654,-0.155334
4,5,01:00:00,7,467,466,38.336,1,30.0,51.530289,-0.228343,51.530197,-0.227949


In [46]:
test.head()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,6,00:00:00,3,153,487,2,30.0,51.465182,-0.255240,51.465204,-0.254595
1,1,6,00:00:00,18,342,343,2,30.0,51.458973,-0.141882,51.459254,-0.141646
2,2,6,00:00:00,8,351,131,0,20.0,51.449437,-0.160688,51.449295,-0.161060
3,3,6,00:00:00,18,68,243,2,30.0,51.457325,-0.143228,51.457466,-0.143110
4,4,6,00:00:00,14,241,408,2,30.0,51.467314,-0.112842,51.466309,-0.113371


Mengubah Time

In [47]:
def remove_time(data):
    data['time'] = data['time'].astype(str).str.replace(':00:00', '')
    data['time'] = np.array(data['time']).astype(np.int64)
    return data

In [48]:
train = remove_time(train)
test = remove_time(test)

In [49]:
train.head()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,5,1,19,102,345,29.126,2,30.0,51.434928,-0.161176,51.434975,-0.161090
1,5,1,12,378,279,46.576,1,30.0,51.623990,-0.176398,51.623811,-0.176424
2,5,1,8,133,390,36.587,0,20.0,51.452530,-0.152437,51.452749,-0.151896
3,5,1,16,329,423,34.063,1,20.0,51.465851,-0.154188,51.465654,-0.155334
4,5,1,7,467,466,38.336,1,30.0,51.530289,-0.228343,51.530197,-0.227949


In [50]:
test.head()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon
0,0,6,0,3,153,487,2,30.0,51.465182,-0.255240,51.465204,-0.254595
1,1,6,0,18,342,343,2,30.0,51.458973,-0.141882,51.459254,-0.141646
2,2,6,0,8,351,131,0,20.0,51.449437,-0.160688,51.449295,-0.161060
3,3,6,0,18,68,243,2,30.0,51.457325,-0.143228,51.457466,-0.143110
4,4,6,0,14,241,408,2,30.0,51.467314,-0.112842,51.466309,-0.113371


Mengubah Lattitude dan Longitude

In [51]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers

    delta_lat = math.radians(lat2 - lat1)
    delta_lon = math.radians(lon2 - lon1)

    a = math.sin(delta_lat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [52]:
train['distance_km'] = train.apply(lambda row: haversine_distance(row['mulai_lat'], row['mulai_lon'], row['akhir_lat'], row['akhir_lon']), axis=1)
test['distance_km'] = test.apply(lambda row: haversine_distance(row['mulai_lat'], row['mulai_lon'], row['akhir_lat'], row['akhir_lon']), axis=1)

In [53]:
train.head()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon,distance_km
0,5,1,19,102,345,29.126,2,30.0,51.434928,-0.161176,51.434975,-0.161090,0.007914
1,5,1,12,378,279,46.576,1,30.0,51.623990,-0.176398,51.623811,-0.176424,0.019985
2,5,1,8,133,390,36.587,0,20.0,51.452530,-0.152437,51.452749,-0.151896,0.044685
3,5,1,16,329,423,34.063,1,20.0,51.465851,-0.154188,51.465654,-0.155334,0.082340
4,5,1,7,467,466,38.336,1,30.0,51.530289,-0.228343,51.530197,-0.227949,0.029118


In [54]:
test.head()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,mulai_lat,mulai_lon,akhir_lat,akhir_lon,distance_km
0,0,6,0,3,153,487,2,30.0,51.465182,-0.255240,51.465204,-0.254595,0.044729
1,1,6,0,18,342,343,2,30.0,51.458973,-0.141882,51.459254,-0.141646,0.035246
2,2,6,0,8,351,131,0,20.0,51.449437,-0.160688,51.449295,-0.161060,0.030265
3,3,6,0,18,68,243,2,30.0,51.457325,-0.143228,51.457466,-0.143110,0.017682
4,4,6,0,14,241,408,2,30.0,51.467314,-0.112842,51.466309,-0.113371,0.117667


Drop column yang tidak diperlukan

In [55]:
columns_to_drop = ['mulai_lat', 'mulai_lon', 'akhir_lat', 'akhir_lon']

# Drop the specified columns from the DataFrame
train.drop(columns=columns_to_drop, inplace=True)
test.drop(columns=columns_to_drop, inplace=True)

In [56]:
train.head()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,highway,max_speed,distance_km
0,5,1,19,102,345,29.126,2,30.0,0.007914
1,5,1,12,378,279,46.576,1,30.0,0.019985
2,5,1,8,133,390,36.587,0,20.0,0.044685
3,5,1,16,329,423,34.063,1,20.0,0.082340
4,5,1,7,467,466,38.336,1,30.0,0.029118


In [57]:
test.head()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km
0,0,6,0,3,153,487,2,30.0,0.044729
1,1,6,0,18,342,343,2,30.0,0.035246
2,2,6,0,8,351,131,0,20.0,0.030265
3,3,6,0,18,68,243,2,30.0,0.017682
4,4,6,0,14,241,408,2,30.0,0.117667


Drop row null pada train

In [58]:
print(train.shape)
train.dropna(inplace=True)

# Reset index dilakukan supaya saat data di iterasi mengunakan index tidak menimbulkan error
train.reset_index(drop=True, inplace=True)
print(train.shape)

(398648, 9)
(387075, 9)


Remove outlier rerata_kecepatan pada train

In [59]:
def DetectOutlierSTDFromY(xtrain, ytrain):
    mean = ytrain.mean(axis=0)
    std = ytrain.std(axis=0)
    lb, ub = mean - 3 * std, mean + 3 * std
    
    # Calculate the boolean mask for filtering
    status = (ytrain >= lb) & (ytrain <= ub)

    # Apply the boolean mask to filter both xtrain and ytrain
    xtrain_filtered = xtrain[status]
    ytrain_filtered = ytrain[status]

    return xtrain_filtered, ytrain_filtered

In [60]:
train['id'] = range(1, len(train) + 1)

In [61]:
X_plot = train.drop('rerata_kecepatan', axis=1)
y_plot = train['rerata_kecepatan']

In [62]:
print(X_plot.shape)

(387075, 9)


In [63]:
X_no_outlier = pd.DataFrame(columns=X_plot.columns)
y_no_outlier = pd.DataFrame(columns=[y_plot.name])

In [64]:
for i in range(20):
    selected_df = train.loc[train['id_jalan'] == i]

    # Filter X and y based on id_jalan
    X_selected = selected_df.drop('rerata_kecepatan', axis=1)
    y_selected = selected_df['rerata_kecepatan']

    X_after, y_after = DetectOutlierSTDFromY(X_selected, np.array(y_selected))

    # Concatenate the filtered data
    X_no_outlier = pd.concat([X_no_outlier, pd.DataFrame(X_after, columns=X_selected.columns)], ignore_index=True)
    y_no_outlier = pd.concat([y_no_outlier, pd.Series(y_after, name=y_selected.name)])


In [65]:
print(X_no_outlier.shape)

(382669, 9)


In [66]:
y_no_outlier = y_no_outlier.drop('rerata_kecepatan', axis=1)
y_no_outlier = y_no_outlier.rename(columns={0:'rerata_kecepatan'})

In [67]:
X_no_outlier.reset_index(drop=True, inplace=True)
y_no_outlier.reset_index(drop=True, inplace=True)

In [68]:
# Combine X_no_outlier and y_no_outlier DataFrames
combined_df = pd.concat([X_no_outlier, y_no_outlier], axis=1)

# Sort by the 'id' column
combined_df = combined_df.sort_values(by='id')

In [69]:
# Drop the 'id' column
combined_df = combined_df.drop('id', axis=1)

In [70]:
X = combined_df.drop('rerata_kecepatan', axis=1)
y = combined_df['rerata_kecepatan']

In [71]:
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

X_df = pd.DataFrame(X, columns=['day', 'time', 'id_jalan', 'id_titik_mulai', 'id_titik_akhir', 'highway', 'max_speed', 'distance_km'])
y_df = pd.DataFrame(y, columns=['rerata_kecepatan'])  

# Concatenate X and y along the columns axis
train = pd.concat([X_df, y_df], axis=1)

In [72]:
train.head()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km,rerata_kecepatan
0,5,1,19,102,345,2,30.0,0.007914,29.126
1,5,1,12,378,279,1,30.0,0.019985,46.576
2,5,1,8,133,390,0,20.0,0.044685,36.587
3,5,1,16,329,423,1,20.0,0.082340,34.063
4,5,1,7,467,466,1,30.0,0.029118,38.336


Mengisi data null pada test

In [73]:
mean_km = []
for i in range(20):
    selected_mean = train.loc[(train['id_jalan'] == i)]['distance_km'].mean()
    mean_km.append(selected_mean)

In [74]:
print(mean_km)

[0.03690426535716202, 0.07076967977717512, 0.04542869144476667, 0.04876279182909531, 0.03692817233329366, 0.036210918007901445, 0.050429093433684576, 0.06921353868218354, 0.04186311987456228, 0.0457299416242689, 0.04850828203518937, 0.059213257285794234, 0.03644480189972879, 0.049887833589681005, 0.06179884392662668, 0.03655751808304448, 0.03854700959792484, 0.038542389649561644, 0.037944438364932186, 0.03547478187922327]


In [75]:
print(test.isnull().sum())

id                   0
day                  0
time                 0
id_jalan             0
id_titik_mulai       0
id_titik_akhir       0
highway              0
max_speed            0
distance_km       3685
dtype: int64


In [76]:
for i in range(20):
    test.loc[(test['id_jalan'] == i) & (test['distance_km'].isnull()), 'distance_km'] = mean_km[i]

In [77]:
print(test.isnull().sum())

id                0
day               0
time              0
id_jalan          0
id_titik_mulai    0
id_titik_akhir    0
highway           0
max_speed         0
distance_km       0
dtype: int64


In [78]:
test = pd.DataFrame(test, columns=['id', 'day', 'time', 'id_jalan','id_titik_mulai','id_titik_akhir','highway', 'max_speed', 'distance_km'])

In [79]:
test.head()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km
0,0,6,0,3,153,487,2,30.0,0.044729
1,1,6,0,18,342,343,2,30.0,0.035246
2,2,6,0,8,351,131,0,20.0,0.030265
3,3,6,0,18,68,243,2,30.0,0.017682
4,4,6,0,14,241,408,2,30.0,0.117667


Scaling

In [80]:
scaler = StandardScaler()

columns_to_scale_train = train.columns[:-1]
columns_to_scale_test = test.columns[1:]

In [81]:
train[columns_to_scale_train] = scaler.fit_transform(train[columns_to_scale_train])
test[columns_to_scale_test] = scaler.transform(test[columns_to_scale_test])

In [82]:
train.head()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km,rerata_kecepatan
0,0.85708,-1.606475,1.655615,-1.043735,0.740123,1.075941,0.753407,-1.064817,29.126
1,0.85708,-1.606475,0.437242,0.981634,0.255519,-0.346031,0.753407,-0.727222,46.576
2,0.85708,-1.606475,-0.258970,-0.816248,1.070534,-1.768003,-1.327304,-0.036389,36.587
3,0.85708,-1.606475,1.133455,0.622057,1.312836,-0.346031,-1.327304,1.016810,34.063
4,0.85708,-1.606475,-0.433024,1.634742,1.628563,-0.346031,0.753407,-0.471775,38.336


In [83]:
test.head()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km
0,0,1.352789,-1.749679,-1.129237,-0.669482,1.782755,1.075941,0.753407,-0.035156
1,1,1.352789,-1.749679,1.481562,0.717455,0.725438,1.075941,0.753407,-0.300382
2,2,1.352789,-1.749679,-0.258970,0.783500,-0.831169,-1.768003,-1.327304,-0.439679
3,3,1.352789,-1.749679,1.481562,-1.293237,-0.008811,1.075941,0.753407,-0.791616
4,4,1.352789,-1.749679,0.785349,-0.023712,1.202699,1.075941,0.753407,2.004848


In [84]:
train.describe()

,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km,rerata_kecepatan
count,3.826690e+05,3.826690e+05,3.826690e+05,3.826690e+05,3.826690e+05,3.826690e+05,3.826690e+05,3.826690e+05,382669.000000
mean,-6.892470e-17,-4.753428e-17,7.754029e-17,-4.690296e-17,1.218066e-17,1.688952e-16,1.480897e-16,-3.813140e-16,35.973082
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,7.708056
min,-1.621462e+00,-1.749679e+00,-1.651396e+00,-1.792240e+00,-1.793034e+00,-1.768003e+00,-1.327304e+00,-1.238240e+00,1.062000
25%,-6.300452e-01,-7.472490e-01,-7.811301e-01,-8.456008e-01,-8.458538e-01,-3.460313e-01,-1.327304e+00,-7.059832e-01,31.738000
50%,-1.343366e-01,1.119767e-01,8.913592e-02,-2.371218e-02,-2.349566e-02,-3.460313e-01,7.534067e-01,-1.861170e-01,36.627000
75%,8.570804e-01,8.279982e-01,7.853487e-01,8.422062e-01,8.429173e-01,1.075941e+00,7.534067e-01,4.567679e-01,41.235000
max,1.352789e+00,1.544020e+00,1.655615e+00,1.781507e+00,1.782755e+00,1.075941e+00,7.534067e-01,6.289441e+00,60.291000


In [85]:
test.describe()

,id,day,time,id_jalan,id_titik_mulai,id_titik_akhir,highway,max_speed,distance_km
count,127489.000000,127489.000000,127489.000000,127489.000000,127489.000000,127489.000000,127489.000000,127489.000000,127489.000000
mean,63744.000000,-0.039806,-0.002008,-0.015285,-0.001659,-0.001503,-0.000814,0.013263,0.010121
std,36803.048572,0.998785,0.995388,1.006175,1.007426,1.008869,0.996016,0.996103,0.991353
min,0.000000,-1.621462,-1.749679,-1.651396,-1.792240,-1.793034,-1.768003,-1.327304,-1.238240
25%,31872.000000,-1.125754,-0.747249,-0.955183,-0.860277,-0.860539,-0.346031,-1.327304,-0.688655
50%,63744.000000,-0.134337,-0.031228,-0.084917,-0.023712,-0.023496,-0.346031,0.753407,-0.186117
75%,95616.000000,0.857080,0.827998,0.785349,0.856883,0.857602,1.075941,0.753407,0.476943
max,127488.000000,1.352789,1.544020,1.655615,1.781507,1.782755,1.075941,0.753407,6.289441


## Modeling

In [86]:
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X, y)

RandomForestRegressor(random_state=42)

## Prediksi

In [87]:
answer = pd.DataFrame()
X = np.array(test.iloc[:, 1:])
answer['id'] = np.array(test.iloc[:, 0])
answer['rerata_kecepatan'] = model.predict(X)

In [88]:
answer.head()

,id,rerata_kecepatan
0,0,43.176099
1,1,38.315288
2,2,37.076574
3,3,44.519582
4,4,27.867527


In [89]:
answer.describe()

,id,rerata_kecepatan
count,127489.000000,127489.000000
mean,63744.000000,35.852013
std,36803.048572,7.358522
min,0.000000,4.251585
25%,31872.000000,31.689302
50%,63744.000000,36.406084
75%,95616.000000,40.916913
max,127488.000000,56.031414


In [90]:
#Jalankan apabila ingin menyimpan hasil prediksi
answer.to_csv('./answer.csv', index=False)

## Evaluasi

In [91]:
X = np.array(train.iloc[:, :-1])

true = train.iloc[:, -1]
pred = model.predict(X)

mse = mean_squared_error(true, pred)
mae = mean_absolute_error(true, pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

Mean Squared Error: 6.223467012118513
Mean Absolute Error: 1.7503251177325638


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eeef503c-21ca-439c-89e9-76d4e91a02f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>